# How to use nodejs simple-imap?

How to log in to Gmail using imap-simple/node IMAP?

How to temporarily allow less secure apps?

https://myaccount.google.com/lesssecureapps

TODO: convert to selenium/sikuli script



How to connect to Gmail using simple-imap?

How do I connect using simple-imap and {credentials}?

node imap client?



In [ ]:
var fs = require('fs');
var path = require('path');
var imaps = require('imap-simple');
var mime = require('mime');
var importer = require('../Core');
var getCredentials = importer.import('decrypt passwords');

var output = path.join(process.cwd(), 'output');
var credentials = getCredentials('accounts.google.com');
process.env.GOOGLE_USER = credentials.Email;
process.env.GOOGLE_PASS = credentials.Passwd;

var config = {
    imap: {
        user: process.env.GOOGLE_USER,
        password: process.env.GOOGLE_PASS,
        host: 'imap.gmail.com',
        port: 993,
        tls: true,
        autotls: 'required',
        authTimeout: 3000
    }
};

function imapClient() {
    var connection;
    return imaps.connect(config)
        .then((conn) => {
            connection = conn;
            return conn.openBox('INBOX');
        })
        .then(() => connection)
}
module.exports = imapClient;


How to search for messages using simple-imap?

How to search for messages in the last few {days}?

How to search for messages {from} a sender's email address?


In [ ]:
var importer = require('../Core');
var imapClient = importer.import('node imap client');

var connection;
function searchImap(from = '*', subject = '*', days = 7) {
    // Fetch emails from the last number of {days}
    var delay = 24 * 3600 * 1000 * days;
    var lastWeek = new Date();
    lastWeek.setTime(Date.now() - delay);
    lastWeek = lastWeek.toISOString();
    var searchCriteria = ['SEEN', ['SINCE', lastWeek]];
    if(from !== '*') {
        searchCriteria.push(['FROM', from])
    }
    if(subject !== '*') {
        searchCriteria.push(['SUBJECT', subject])
    }
    var fetchOptions = {bodies: ['HEADER.FIELDS (FROM TO SUBJECT DATE)'], struct: true};
    var messages = [];
    return imapClient()
        .then(r => connection = r)
        .then(() => connection.search(searchCriteria, fetchOptions))
        .then(m => {
            messages = m;
            return m;
        })
}
module.exports = searchImap;

if(typeof $$ !== 'undefined') {
$$.async();
searchImap('*', 'megamind')
    .then(r => $$.sendResult(r))
    .catch((e) => $$.sendError(e))
}


How to list the subjects from simple-imap messages?

How to list the sender's email address from simple-imap messages?


In [ ]:
htmlPrint = '';
messages.forEach(message => {
    var header = message.parts.filter(function (part) {
        return part.which.indexOf('HEADER') > -1;
    });
    var subject = header[0].body.subject[0];
    var from = header[0].body.from[0];
    htmlPrint += '<li>subject: ' + subject + ', from: ' + from + '</li>\n';
});
$$.mime({'text/markdown': 'Usage:\n\n```html\n' + htmlPrint + '\n```\nOuput:\n'});



How to extract a list of image attachments from an email?

How do I extract attachments from {message}

How to get attachments using simple-imap?

How to get attachements from the last few {days}?

How to get messages {from} and email address?



In [ ]:
$$.async();
attachments = []
Promise.all(messages.map(message => {
    return Promise.all(imaps.getParts(message.attributes.struct)
        .filter((part) => part.disposition && part.disposition.type === 'ATTACHMENT')
        // retrieve the attachments only of the messages with attachments 
        .map((part) => connection.getPartData(message, part).then((partData) => ({
            filename: part.disposition.params.filename,
            data: partData
        }))))
        .then(a => attachments = attachments.concat(a));
})).then((o) => $$.done(attachments)).catch(e => $$.done(e));

How to save an attachments from simple-imap?

How to save a {file}?

How to save binary {data}?

How to {output} all attachments in the requested {format}?



In [ ]:
// TODO: convert the attachments to a virtual filesystem, lay it on top of the current app, run end-to-end tests
if (!fs.existsSync(output)) {
    fs.mkdirSync(output);
}
var result = attachments.map((attachment) => {
    return new Promise((resolve, reject) => fs.writeFile(
        path.join(output, attachment.filename),
        attachment.data,
        'binary',
        function (err) {
            if (err) reject(err);
            else resolve(attachment);
        }));
});
$$.async();
Promise.all(result).then(images => {
    html = '';
    htmlPrint = '';
    images.forEach((i) => {
        var filename = i.filename.split('/').pop();
        var ext = mime.lookup(i.filename);
        html += '<img src="data:' + ext + ';base64,' + (new Buffer(i.data, 'binary')).toString('base64') + '" />';
        htmlPrint += '<li><img src="/assets/' + filename + '" /></li>\n';
    });
    $$.mime({'text/markdown': 'Usage:\n\n```html\n' + htmlPrint + '\n```\nOutput:\n' + html});
}).catch(e => $$.done(e));

